# Monk2
Grid Search for Monk2 dataset model selection


In [ ]:
from exclusiveAI.ConfiguratorGen import ConfiguratorGen
from exclusiveAI.datasets.monk import read_monk2
from exclusiveAI.components.Validation import *
from exclusiveAI.utils import one_hot_encoding
from exclusiveAI.components.CallBacks import *
from exclusiveAI.Composer import Composer
import pandas as pd
import numpy as np

#### Read Monk2 dataset 

In [ ]:
training_data, training_labels, test_data, test_labels = read_monk2()

One-Hot Encoding Training Dataset

In [ ]:
training_data = one_hot_encoding(training_data)

One-Hot Encoding Test Dataset

In [ ]:
test_data = one_hot_encoding(test_data)

Early Stopping Callback with 50 epochs patience limit

In [ ]:
patience_limit = 50
early_stopping = EarlyStoppingCallback(patience_limit=patience_limit)

Configuration Generator Grid Search for better hyperparameters selection

In [ ]:
regularizations = np.arange(0, 0.5, 0.01)
learning_rates = np.arange(0.01, 0.4, 0.01)
number_of_units = range(1, 10, 1)
number_of_layers = range(1, 3, 1) 
initializers = ["uniform", "gaussian"]
momentums = np.arange(0, 0.999, 0.001)
activations = ["sigmoid"]

myConfigurator = ConfiguratorGen(random=False, regularizations=regularizations, learning_rates=learning_rates,
                                 loss_functions=['mse'], optimizer=['sgd'],
                                 activation_functions=activations,
                                 number_of_units=number_of_units, number_of_layers=number_of_layers,
                                 momentums=momentums, initializers=initializers, input_shapes=training_data.shape, verbose=False,
                                 callbacks=[early_stopping], output_activation='sigmoid'
                                 )

HoldOut validation with our configurations

In [ ]:
myValidation = HoldOut(models=myConfigurator, input=training_data, target=training_labels, debug=True)
configs = myValidation.hold_out(all_models=True)

In [ ]:
configs = pd.DataFrame(configs)
# Save as json
configs.to_json('monk2_configurations.json')